<a href="https://colab.research.google.com/github/gaurav-kr92/Omini_channel/blob/main/Omnichannel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies

In [ ]:
!pip install mesa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import mesa
import random
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline 

# Resources Class

In [ ]:
def generate_value_matrix(n,m):
  return np.random.randint(10, size=(n,m))


In [ ]:
def generate_cost_matrix(n,m):
  return np.random.randint(3,size=(n,m))


In [213]:
def ispossible_switch(stage_no,current_channel,to_channel,value,cost,integration_factor,switching_matrix):
  '''
  Function used in scenario-2;
  return bool, whether it is possible to change channel in next stage; 
  '''
  # base condition( if current_channel is same to_channel we return false;)
  if current_channel == to_channel :
   return False

  # calculate what will be dicission if no switch occur and same channel used for next stage; 
  without_switch_decission = value[stage_no][current_channel] - cost[stage_no][current_channel]

  # let assume there is switching, then decission (value and cost ) will be fxn of integration factor and switching matrix; 
  with_switch_decission = value[stage_no][to_channel] - cost[stage_no][to_channel]
  # Now taking accoount for integration factor and switching cost; 
  with_switch_decission = with_switch_decission + switching_matrix[current_channel][to_channel] - integration_factor[current_channel][to_channel]

  # if we found that after switching decision decreases then switch it , return true ;
  if with_switch_decission < without_switch_decission:
    return True
  else:
    return False  


  

#Scenario No -1

In [239]:
def scenario_1(value,cost):
  '''
  scenario 1 - 
  Only one channel to used across all n stages
  '''

  
  # summing value for each each stages 
  sum_of_value_across_each_channel = np.sum(value,axis=0)
  # summing cost for each of stages
  sum_of_cost_across_each_channel = np.sum(cost,axis=0)
  
  # defined decission array , which contain decission for each channel 
  decision_for_each_channel = sum_of_value_across_each_channel - sum_of_cost_across_each_channel
  print("Decision for each Channel : \n" ,decision_for_each_channel)

  #Finding which channel yeild minimum decision 
  minElement = np.amin(decision_for_each_channel)
  print("Minimun decision : ", minElement)
  # Get the indices of minimum element in numpy array
  result = np.where(decision_for_each_channel == np.amin(decision_for_each_channel))

  print('List of Channel through which we got minimum decision :', result[0]+1)

# Scenario 2

In [241]:
def scenario_2(no_of_channels,no_of_stages,value,cost):
  '''
  Scenario_2 : There is switch from within Channel across buying stages; 
  When switch occur both integration factor and switching cost are used to calcualte the value and cost; 
  '''
  # Create a integration facator matrix : it will be 2-D array of size(no_of_channel * no_of_channel) with diagonal element as 1; 
  integration_factor = np.random.rand(no_of_channels,no_of_channels)
  np.fill_diagonal(integration_factor,1)
  print("Integration factor : \n", integration_factor)
 

  # Create a switching matrix : it will be 2-D array of size(no_of_channel * no_of_channel) with diagonal element as 0; 
  switching_matrix = np.random.rand(no_of_channels,no_of_channels)
  np.fill_diagonal(switching_matrix,0)
  print("Switching_matrix : \n", switching_matrix)

  minimum_decision = 1e9
  path_taken= []

  # Taking 100 trials , and assinging a starting channel randomly;
  for i in range(100):
    # For each trials, randomly chosing starting channel( i.e for stages 1 which channel to select)
    start_channel = random.randint(0,no_of_channels-1)
    current_channel = start_channel 

    # Now we have to switch channel for next stage of journey, But how many switches we performed in overall customer journey process. 
    # Let it generate random no of switches (ranges for 1 to no_of_stages)
    no_of_switches = random.randint(1,no_of_stages-1) 

    count =0; 
   
    path = [current_channel]
    decision = value[0][start_channel] - cost[0][start_channel]

   # Iterating through all stage of customer journey ; 
    for stage in range(no_of_stages):
      if stage==0: continue
      # Generate a random channel no , to which it switches form current channel if it is possible; 
      to_channel = random.randint(0,no_of_channels-1)
      if count < no_of_switches and ispossible_switch(stage,current_channel,to_channel,value,cost,integration_factor,switching_matrix):
        decision += value[stage][to_channel] - cost[stage][to_channel] + switching_matrix[current_channel][to_channel] - integration_factor[current_channel][to_channel]
        count +=1
        path.append(to_channel)
        current_channel = to_channel 

      else:
        path.append(current_channel)
        decision += value[stage][current_channel] - cost[stage][current_channel]

    if minimum_decision > decision:
      minimum_decision = decision
      path_taken = path

  print("Minimum decision in 100 iteration : ", minimum_decision)
  print("Channels used in this journey is : \n" ,path_taken)   




# Agent class

In [ ]:
class Agent(mesa.Agent):
  def __init__(self,unique_id,model,pos):
    super().__init__(unique_id,model)
    self.pos = pos
    


# Model class

In [237]:
class omnichannel(mesa.Model):

  def __init__(self, no_of_agents = random.randint(0, 9),
               no_of_stages = random.randrange(2, 8), 
               no_of_channels = random.randrange(2,20)):

    # Intialise no of stages and no of channel to given value else random valve; 
    self.no_of_stages = no_of_stages
    self.no_of_channels = no_of_channels
    self.no_of_agents = no_of_agents

    # Intiate Mesa grid class ( A Row denoting  = a stage and Coloum denoting =  a channel)
    # width will store no_of_channel;
    # height will store no of stages;
    self.width = self.no_of_channels +1 
    self.height = self.no_of_stages +1
    self.grid = mesa.space.MultiGrid(self.width,self.height, torus = False)
    #Intiate scheduler
    self.schedule =mesa.time.RandomActivationByType(self)


     #Printing the intial given constraints;
    print("Number of buying stages : ",no_of_stages)
    print("Number of Channel available: ",no_of_channels)

    # generate a random value matrix 
    value = generate_value_matrix(no_of_stages,no_of_channels)
    print("Value matrix: \n", value)

    # generate random cost matrix
    cost = generate_cost_matrix(no_of_stages,no_of_channels)
    print("Cost Matrix: \n",cost)

    agent_id =0 
    
   
    for i in range(self.no_of_agents):
      # Here we are assuming each agent start their purchase journey from stage 1. 
      # So in grid , intial position will be (0,y)
      x = random.randint(0,self.no_of_channels)
      agent = Agent(agent_id,self,(x,0))
      self.grid.place_agent(agent,(x,0))
      self.schedule.add(agent)
      agent_id +=1


    # Scenerio One; ( No change of channel And only one Agent )
    # Outputing minimun decision and channel used; 
    print("Scenerio 1 result : \n")
    scenario_1(value,cost)

    # scenario Two; (Change in channel in upcoming stages)
    print("\n Scenerio 2 result : \n")
    scenario_2(no_of_channels,no_of_stages,value,cost)




In [242]:
model = omnichannel()
model

Number of buying stages :  7
Number of Channel available:  19
Value matrix: 
 [[2 7 0 6 2 7 2 5 8 1 9 0 8 7 0 8 2 3 9]
 [1 4 5 7 3 8 4 4 1 6 4 0 7 7 7 7 3 5 3]
 [9 6 8 7 6 5 1 5 4 3 3 7 2 5 0 2 3 6 7]
 [5 5 4 2 5 7 6 0 7 1 6 4 9 4 3 1 1 2 6]
 [1 5 1 8 1 8 6 7 5 0 8 7 6 8 6 4 5 4 2]
 [6 6 2 3 5 8 3 1 4 1 5 6 4 3 3 5 0 8 8]
 [5 1 9 1 4 0 6 8 1 9 2 8 3 8 6 7 7 7 1]]
Cost Matrix: 
 [[2 0 1 1 2 1 2 1 0 0 1 2 0 2 1 2 2 2 1]
 [1 1 0 2 0 0 2 2 0 0 0 2 0 1 0 1 1 2 1]
 [0 1 1 1 2 1 2 2 1 0 0 0 0 1 0 1 1 0 2]
 [2 1 2 1 0 0 2 1 2 2 1 1 2 0 0 1 2 0 2]
 [2 1 1 0 0 0 2 2 1 1 1 2 0 1 0 1 0 0 0]
 [0 2 2 1 1 2 0 1 2 2 2 0 2 2 2 1 0 2 1]
 [2 1 1 0 1 2 0 1 0 2 0 2 1 2 1 2 2 1 1]]
Scenerio 1 result : 

Decision for each Channel : 
 [20 27 21 28 20 37 18 20 24 14 32 23 34 33 21 25 13 28 28]
Minimun decision :  13
List of Channel through which we got minimum decision : [17]

 Scenerio 2 result : 

Integration factor : 
 [[1.00000000e+00 9.77214171e-01 3.95697812e-01 9.45181999e-01
  2.75619211e-01 3.12372809